<a href="https://colab.research.google.com/github/dftow01/Data-Mining/blob/main/Apriori-and-FP-growth-implementations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apyori

### Import the libraries

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
import pandas as pd
import time

### Import Dataset

In [ ]:
arm_data = pd.read_csv('/content/drive/MyDrive/LAB1-ARM-Dataset.csv')
arm_data.head(18)

,Transaction ID,Bread,Milk,Diaper,Beer,Cola,Chicken,Eggs,Apple,Juice,Banana
0,1,1,1,0,1,0,0,1,0,0,1
1,2,1,0,1,1,1,0,1,1,0,0
2,3,1,1,1,0,1,1,0,1,1,0
3,4,0,1,1,1,0,1,1,0,1,0
4,5,1,1,1,0,1,0,0,1,0,1
5,6,1,1,1,0,0,1,1,1,0,0
6,7,1,0,1,1,1,1,0,0,1,0
7,8,0,1,0,1,0,1,1,1,0,0
8,9,1,0,1,0,1,0,1,0,1,0
9,10,1,1,0,0,1,1,0,1,1,0


### Data Manipulation
- Changes the 1 & 0 to the column names

In [ ]:
for i in arm_data:
  arm_data[i] = arm_data[i].apply(lambda x: i if x==1 else "")

In [ ]:
arm_data.head(15)

,Transaction ID,Bread,Milk,Diaper,Beer,Cola,Chicken,Eggs,Apple,Juice,Banana
0,Transaction ID,Bread,Milk,,Beer,,,Eggs,,,Banana
1,,Bread,,Diaper,Beer,Cola,,Eggs,Apple,,
2,,Bread,Milk,Diaper,,Cola,Chicken,,Apple,Juice,
3,,,Milk,Diaper,Beer,,Chicken,Eggs,,Juice,
4,,Bread,Milk,Diaper,,Cola,,,Apple,,Banana
5,,Bread,Milk,Diaper,,,Chicken,Eggs,Apple,,
6,,Bread,,Diaper,Beer,Cola,Chicken,,,Juice,
7,,,Milk,,Beer,,Chicken,Eggs,Apple,,
8,,Bread,,Diaper,,Cola,,Eggs,,Juice,
9,,Bread,Milk,,,Cola,Chicken,,Apple,Juice,


### Remove the Transaction ID Column

In [ ]:
arm_data.drop('Transaction ID', axis=1,inplace=True)

In [ ]:
arm_data

,Bread,Milk,Diaper,Beer,Cola,Chicken,Eggs,Apple,Juice,Banana
0,Bread,Milk,,Beer,,,Eggs,,,Banana
1,Bread,,Diaper,Beer,Cola,,Eggs,Apple,,
2,Bread,Milk,Diaper,,Cola,Chicken,,Apple,Juice,
3,,Milk,Diaper,Beer,,Chicken,Eggs,,Juice,
4,Bread,Milk,Diaper,,Cola,,,Apple,,Banana
5,Bread,Milk,Diaper,,,Chicken,Eggs,Apple,,
6,Bread,,Diaper,Beer,Cola,Chicken,,,Juice,
7,,Milk,,Beer,,Chicken,Eggs,Apple,,
8,Bread,,Diaper,,Cola,,Eggs,,Juice,
9,Bread,Milk,,,Cola,Chicken,,Apple,Juice,


In [ ]:
transaction = arm_data.apply(lambda row: [item for item in row if item != ""], axis=1).tolist()

In [ ]:
transaction

[['Bread', 'Milk', 'Beer', 'Eggs', 'Banana'],
 ['Bread', 'Diaper', 'Beer', 'Cola', 'Eggs', 'Apple'],
 ['Bread', 'Milk', 'Diaper', 'Cola', 'Chicken', 'Apple', 'Juice'],
 ['Milk', 'Diaper', 'Beer', 'Chicken', 'Eggs', 'Juice'],
 ['Bread', 'Milk', 'Diaper', 'Cola', 'Apple', 'Banana'],
 ['Bread', 'Milk', 'Diaper', 'Chicken', 'Eggs', 'Apple'],
 ['Bread', 'Diaper', 'Beer', 'Cola', 'Chicken', 'Juice'],
 ['Milk', 'Beer', 'Chicken', 'Eggs', 'Apple'],
 ['Bread', 'Diaper', 'Cola', 'Eggs', 'Juice'],
 ['Bread', 'Milk', 'Cola', 'Chicken', 'Apple', 'Juice'],
 ['Diaper', 'Cola', 'Eggs', 'Apple', 'Juice', 'Banana'],
 ['Bread', 'Milk', 'Diaper', 'Chicken', 'Eggs', 'Juice'],
 ['Bread', 'Diaper', 'Beer', 'Cola', 'Apple', 'Juice'],
 ['Milk', 'Diaper', 'Beer', 'Chicken', 'Apple', 'Banana'],
 ['Bread', 'Milk', 'Cola', 'Chicken', 'Eggs', 'Banana']]

In [ ]:
oht = TransactionEncoder()
oht_ary = oht.fit(transaction).transform(transaction)
df = pd.DataFrame(oht_ary, columns=oht.columns_)


In [ ]:
print(df)

    Apple  Banana   Beer  Bread  Chicken   Cola  Diaper   Eggs  Juice   Milk
0   False    True   True   True    False  False   False   True  False   True
1    True   False   True   True    False   True    True   True  False  False
2    True   False  False   True     True   True    True  False   True   True
3   False   False   True  False     True  False    True   True   True   True
4    True    True  False   True    False   True    True  False  False   True
5    True   False  False   True     True  False    True   True  False   True
6   False   False   True   True     True   True    True  False   True  False
7    True   False   True  False     True  False   False   True  False   True
8   False   False  False   True    False   True    True   True   True  False
9    True   False  False   True     True   True   False  False   True   True
10   True    True  False  False    False   True    True   True   True  False
11  False   False  False   True     True  False    True   True   True   True

## Part 1: Apriori Algoprithm

### Frequency Itemset

In [ ]:
start_time = time.time()
frequent_itemsets_ap = apriori(df, min_support=0.3, use_colnames=True)
end_time = time.time()
execution_time_apriori = end_time - start_time
print("Frequent Itemsets:")
print(frequent_itemsets_ap)

Frequent Itemsets:
     support                 itemsets
0   0.600000                  (Apple)
1   0.333333                 (Banana)
2   0.466667                   (Beer)
3   0.733333                  (Bread)
4   0.600000                (Chicken)
5   0.600000                   (Cola)
6   0.733333                 (Diaper)
7   0.600000                   (Eggs)
8   0.533333                  (Juice)
9   0.666667                   (Milk)
10  0.400000           (Bread, Apple)
11  0.333333         (Chicken, Apple)
12  0.400000            (Cola, Apple)
13  0.466667          (Diaper, Apple)
14  0.400000            (Milk, Apple)
15  0.333333           (Beer, Diaper)
16  0.400000         (Chicken, Bread)
17  0.533333            (Cola, Bread)
18  0.533333          (Diaper, Bread)
19  0.400000            (Eggs, Bread)
20  0.400000           (Juice, Bread)
21  0.466667            (Milk, Bread)
22  0.400000        (Chicken, Diaper)
23  0.333333          (Chicken, Eggs)
24  0.333333         (Chicken, 

### Association Rules Mining

In [ ]:
rules = association_rules(frequent_itemsets_ap, metric="confidence", min_threshold=0.6)
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

for _, row in rules.iterrows():
    print(f"\nRule: {set(row['antecedents'])} → {set(row['consequents'])}")
    print(f"Support: {row['support']:.3f}, Confidence: {row['confidence']:.3f}, Lift: {row['lift']:.3f}")



Association Rules:
        antecedents     consequents   support  confidence      lift
0           (Apple)         (Bread)  0.400000    0.666667  0.909091
1            (Cola)         (Apple)  0.400000    0.666667  1.111111
2           (Apple)          (Cola)  0.400000    0.666667  1.111111
3          (Diaper)         (Apple)  0.466667    0.636364  1.060606
4           (Apple)        (Diaper)  0.466667    0.777778  1.060606
..              ...             ...       ...         ...       ...
63  (Chicken, Eggs)          (Milk)  0.333333    1.000000  1.500000
64    (Juice, Cola)        (Diaper)  0.333333    0.833333  1.136364
65  (Juice, Diaper)          (Cola)  0.333333    0.714286  1.190476
66   (Cola, Diaper)         (Juice)  0.333333    0.714286  1.339286
67          (Juice)  (Cola, Diaper)  0.333333    0.625000  1.339286

[68 rows x 5 columns]

Rule: {'Apple'} → {'Bread'}
Support: 0.400, Confidence: 0.667, Lift: 0.909

Rule: {'Cola'} → {'Apple'}
Support: 0.400, Confidence: 0.667, Li

## Part 2: FP-growth

### Frequency Itemset

In [ ]:
start_time = time.time()
frequent_itemsets_fp = fpgrowth(df, min_support=0.3, use_colnames=True)
end_time = time.time()
execution_time_fpgrowth = end_time - start_time
print("FP-growth Frequent Itemsets:")
print(frequent_itemsets_fp)

FP-growth Frequent Itemsets:
     support                 itemsets
0   0.733333                  (Bread)
1   0.666667                   (Milk)
2   0.600000                   (Eggs)
3   0.466667                   (Beer)
4   0.333333                 (Banana)
5   0.733333                 (Diaper)
6   0.600000                   (Cola)
7   0.600000                  (Apple)
8   0.600000                (Chicken)
9   0.533333                  (Juice)
10  0.466667            (Milk, Bread)
11  0.400000           (Milk, Diaper)
12  0.400000             (Milk, Eggs)
13  0.400000            (Eggs, Bread)
14  0.400000           (Eggs, Diaper)
15  0.333333          (Chicken, Eggs)
16  0.333333    (Milk, Chicken, Eggs)
17  0.333333           (Beer, Diaper)
18  0.533333          (Diaper, Bread)
19  0.533333            (Cola, Bread)
20  0.466667           (Cola, Diaper)
21  0.400000    (Diaper, Cola, Bread)
22  0.466667          (Diaper, Apple)
23  0.400000            (Cola, Apple)
24  0.400000         

### Association Rules Mining

In [ ]:
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.6)
print("\nAssociation Rules:")
print(rules_fp[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
for _, row in rules_fp.iterrows():
    print(f"\nRule: {set(row['antecedents'])} → {set(row['consequents'])}")
    print(f"Support: {row['support']:.3f}, Confidence: {row['confidence']:.3f}, Lift: {row['lift']:.3f}")



Association Rules:
        antecedents      consequents   support  confidence      lift
0            (Milk)          (Bread)  0.466667    0.700000  0.954545
1           (Bread)           (Milk)  0.466667    0.636364  0.954545
2            (Milk)         (Diaper)  0.400000    0.600000  0.818182
3            (Milk)           (Eggs)  0.400000    0.600000  1.000000
4            (Eggs)           (Milk)  0.400000    0.666667  1.000000
..              ...              ...       ...         ...       ...
63          (Juice)    (Cola, Bread)  0.333333    0.625000  1.171875
64  (Juice, Diaper)          (Bread)  0.333333    0.714286  0.974026
65  (Bread, Diaper)          (Juice)  0.333333    0.625000  1.171875
66   (Juice, Bread)         (Diaper)  0.333333    0.833333  1.136364
67          (Juice)  (Bread, Diaper)  0.333333    0.625000  1.171875

[68 rows x 5 columns]

Rule: {'Milk'} → {'Bread'}
Support: 0.467, Confidence: 0.700, Lift: 0.955

Rule: {'Bread'} → {'Milk'}
Support: 0.467, Confidence

## Computation Time

In [ ]:
print("--Performance Comparison in terms of computation time")
print(f"Apriori Execution Time: {execution_time_apriori:.4f} seconds")
print(f"FP-Growth Execution Time: {execution_time_fpgrowth:.4f} seconds")

--Performance Comparison in terms of computation time
Apriori Execution Time: 0.0070 seconds
FP-Growth Execution Time: 0.0334 seconds


## Check the number of identical results from the Apriori and FP-growth algorithms

In [ ]:
common_rules = pd.merge(rules, rules_fp, on=['antecedents', 'consequents'], how='inner', suffixes=('_apriori', '_fpgrowth'))
print(len(common_rules))
print("\nAssociation Rules (Apriori and FP-Growth):")
for _, row in common_rules.iterrows():
    print(f"\nRule: {set(row['antecedents'])} → {set(row['consequents'])}")
    print(f"Apriori - Support: {row['support_apriori']:.3f}, Confidence: {row['confidence_apriori']:.3f}, Lift: {row['lift_apriori']:.3f}")
    print(f"FP-Growth - Support: {row['support_fpgrowth']:.3f}, Confidence: {row['confidence_fpgrowth']:.3f}, Lift: {row['lift_fpgrowth']:.3f}")


68

Association Rules (Apriori and FP-Growth):

Rule: {'Apple'} → {'Bread'}
Apriori - Support: 0.400, Confidence: 0.667, Lift: 0.909
FP-Growth - Support: 0.400, Confidence: 0.667, Lift: 0.909

Rule: {'Cola'} → {'Apple'}
Apriori - Support: 0.400, Confidence: 0.667, Lift: 1.111
FP-Growth - Support: 0.400, Confidence: 0.667, Lift: 1.111

Rule: {'Apple'} → {'Cola'}
Apriori - Support: 0.400, Confidence: 0.667, Lift: 1.111
FP-Growth - Support: 0.400, Confidence: 0.667, Lift: 1.111

Rule: {'Diaper'} → {'Apple'}
Apriori - Support: 0.467, Confidence: 0.636, Lift: 1.061
FP-Growth - Support: 0.467, Confidence: 0.636, Lift: 1.061

Rule: {'Apple'} → {'Diaper'}
Apriori - Support: 0.467, Confidence: 0.778, Lift: 1.061
FP-Growth - Support: 0.467, Confidence: 0.778, Lift: 1.061

Rule: {'Milk'} → {'Apple'}
Apriori - Support: 0.400, Confidence: 0.600, Lift: 1.000
FP-Growth - Support: 0.400, Confidence: 0.600, Lift: 1.000

Rule: {'Apple'} → {'Milk'}
Apriori - Support: 0.400, Confidence: 0.667, Lift: 1.000